In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install --upgrade datasets
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install --upgrade evaluate datasets
#!pip install --upgrade transformers huggingface_hub

In [ ]:
!pip install transformers==4.47.1

In [ ]:
import pandas as pd
import os

train_df = pd.read_csv('/content/drive/MyDrive/Twi_ASR/1_Mose_train.csv')
#train_df = train_df.drop(index=[19307, 3547, 3548, 3549, 3550, 3551, 3552, 3553, 3554, 3555, 3556, 3557])
test_df = pd.read_csv('/content/drive/MyDrive/Twi_ASR/1_Mose_test.csv')
#test_df = test_df.drop(index=[1])

In [ ]:
"""train_df = train_df.iloc[1:3545]
test_df = test_df.iloc[846:1852]"""

In [ ]:
"""train_df = train_df.iloc[3546:5039]
test_df = test_df.iloc[0:837]"""

In [ ]:
"""train_df = train_df.iloc[5039:8059]
test_df = test_df.iloc[846:1189]"""

In [ ]:
"""train_df = train_df.iloc[8059:10005]
test_df = test_df.iloc[1189:1411]"""


In [ ]:
"""train_df = train_df.iloc[10005:21237]
test_df = test_df.iloc[1411:2688]"""

In [ ]:
train_df = train_df.iloc[21237:33102]
test_df = test_df.iloc[2688:4033]

In [ ]:
from datasets import load_dataset, Dataset

# Converting pandas DataFrame to Hugging Face Dataset
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)

In [ ]:
test_df.loc[test_df['path'] == 'AsantiTwiMa20-SLWOah2f-Tmp023-GXRJsS.wav' ]

,Unnamed: 0,path,sentence


In [ ]:
from datasets import ClassLabel
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(test.remove_columns(["path"]), num_examples=20)

,sentence
0,Nnipadɔm pa a na
1,"Wɔsoaa no, de no kɔɔ Kanaan asase so"
2,"akyenkyennuro, sɛdeɛ ɛbɛyɛ a, ɔrensɛe"
3,mʼanim ase
4,Benyamin yɛ pataku a ɔyɛ nam
5,Yei yɛ ayi kɛseɛ bi a
6,Kanaan asase so no mu na monkɔsie me
7,kye nʼahaboa we
8,Otumfoɔ a ɔde ɔsoro nhyira bɛhyira wo
9,na ne mman tra afasuo


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\)\(\/\•\’\—\·]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
train = train.map(remove_special_characters)
test = test.map(remove_special_characters)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(train.remove_columns(['path']))

,sentence
0,enti ɔde wɔn brɛɛ onipa sɛ ɔnhwehwɛ edin ko a ɔde bɛtoto wɔn
1,ma ɔdii akɔneaba kɔsii sɛ nsuo no nyinaa we firii asase no so
2,ɛberɛ a aborɔnoma no sane baa ne nkyɛn anwummerɛ no
3,kain de nʼafuom nnɔbaeɛ bi kɔbɔɔ awurade afɔdeɛ
4,hann mmra nkyɛmu a ɛwɔ ewiem no mu
5,ɛnnɛ woapam me afiri asase so
6,na arfaksad woo sela
7,ɔdii mfeɛ
8,aha nwɔtwe
9,ahyɛaseɛ no


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[ń]', 'n', batch["sentence"])
    batch["sentence"] = re.sub('[ĩ]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[á]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[ũ]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[õ]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[ã]', 'a', batch["sentence"])
    return batch

In [ ]:
train = train.map(replace_hatted_characters)
test = test.map(replace_hatted_characters)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train.column_names)
vocab_test = test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test.column_names)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'p': 0,
 'm': 1,
 'b': 2,
 'i': 3,
 'n': 4,
 'y': 5,
 't': 6,
 'h': 7,
 '\u2006': 8,
 'u': 9,
 'd': 10,
 'f': 11,
 'k': 12,
 'a': 13,
 'ʼ': 14,
 'w': 15,
 'ɛ': 16,
 ' ': 17,
 'e': 18,
 'l': 19,
 'o': 20,
 'ɔ': 21,
 'r': 22,
 'g': 23,
 's': 24}

In [ ]:
vocab_dict = {
    'y': 0, 'n': 1, 'g': 2, 'p': 3, 'i': 4, 'j': 5, '0': 6, 'ʹ': 7, "'": 8,
    'ɛ': 9, 'b': 10, 'c': 11, '3': 12, 'р': 13, 'u': 14, 'h': 15, 'a': 16,
    'w': 17, 'm': 18, 'f': 19, ' ': 20, 'а': 21, 'у': 22, 'd': 23, 's': 24,
    'k': 25, 'z': 26, 't': 27, '5': 28, '9': 29, 'l': 30, 'r': 31, 'x': 32,
    '2': 33, 'o': 34, 'q': 35, '́': 36, '4': 37, 'ɔ': 38, 'e': 39, 'v': 40,
    '1': 41, '\u2006': 42, 'ʼ': 43
}
vocab_dict


{'y': 0,
 'n': 1,
 'g': 2,
 'p': 3,
 'i': 4,
 'j': 5,
 '0': 6,
 'ʹ': 7,
 "'": 8,
 'ɛ': 9,
 'b': 10,
 'c': 11,
 '3': 12,
 'р': 13,
 'u': 14,
 'h': 15,
 'a': 16,
 'w': 17,
 'm': 18,
 'f': 19,
 ' ': 20,
 'а': 21,
 'у': 22,
 'd': 23,
 's': 24,
 'k': 25,
 'z': 26,
 't': 27,
 '5': 28,
 '9': 29,
 'l': 30,
 'r': 31,
 'x': 32,
 '2': 33,
 'o': 34,
 'q': 35,
 '́': 36,
 '4': 37,
 'ɔ': 38,
 'e': 39,
 'v': 40,
 '1': 41,
 '\u2006': 42,
 'ʼ': 43}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

46

In [ ]:
import json
with open('/content/drive/MyDrive/Twi_ASR/new_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/Twi_ASR/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Preprocessing Audio Data


In [ ]:
"""from datasets import Dataset, concatenate_datasets

def process_dataset_in_chunks(dataset, chunk_size, fn, cache_file_name):
    num_chunks = len(dataset) // chunk_size + 1
    processed_chunks = []

    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(dataset))
        print(f"Processing chunk {i+1}/{num_chunks}")

        chunk = dataset.select(range(start, end))
        processed_chunk = chunk.map(fn, remove_columns=chunk.column_names, keep_in_memory=False, cache_file_name=cache_file_name, num_proc=1)
        processed_chunks.append(processed_chunk)

    return concatenate_datasets(processed_chunks)

# Process train and test datasets in chunks
chunk_size = 1000  # Adjust based on your RAM capacity
train_cache_file_name = "/content/drive/MyDrive/Twi_ASR/processed_train.arrow"
test_cache_file_name = "/content/drive/MyDrive/Twi_ASR/processed_test.arrow" """


In [ ]:
"""train = process_dataset_in_chunks(train, chunk_size, speech_file_to_array_fn, train_cache_file_name)
test = process_dataset_in_chunks(test, chunk_size, speech_file_to_array_fn, test_cache_file_name)"""

In [ ]:
import torchaudio
import os

# Define base path to the raw_audio folder
base_path = "/content/drive/MyDrive/Twi_ASR/1_Mose_audio_splits"

def speech_file_to_array_fn(batch):

    speech_array, sampling_rate = torchaudio.load(os.path.join(base_path, batch["path"]))
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]

    return batch

In [ ]:
train = train.map(speech_file_to_array_fn, remove_columns=train.column_names, num_proc=1)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

In [ ]:
"""train.save_to_disk("/content/train")"""

In [ ]:
"""# Clear memory
import gc
del train
gc.collect()"""

2111

In [ ]:
"""# Reload the dataset
from datasets import load_from_disk
train = load_from_disk("/content/train")"""

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
test = test.map(speech_file_to_array_fn, remove_columns=test.column_names, num_proc=1)

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
import librosa
import numpy as np

def resample_audio(batch):
    """
    Resample audio data in 'speech' to 16kHz.
    """
    # Ensure 'speech' and 'sampling_rate' exist in batch
    if "speech" in batch and "sampling_rate" in batch:
        audio = np.asarray(batch["speech"])  # Convert speech to NumPy array
        orig_sr = batch["sampling_rate"]     # Original sampling rate

        # Perform resampling
        batch["speech"] = librosa.resample(y=audio, orig_sr=orig_sr, target_sr=16000)
        batch["sampling_rate"] = 16000  # Update sampling rate

    return batch

In [ ]:
train = train.map(resample_audio, num_proc=1)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

In [ ]:
test = test.map(resample_audio, num_proc=1)

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train)-1)

print(train[rand_int]["target_text"])
ipd.Audio(data=np.asarray(train[rand_int]["speech"]), autoplay=True, rate=16000)

ne abɔdeɛ biara a wɔnam asase so 


In [ ]:
rand_int = random.randint(0, len(train))

print("Target text:", train[rand_int]["target_text"])
print("Input array shape:", np.asarray(train[rand_int]["speech"]).shape)
print("Sampling rate:", train[rand_int]["sampling_rate"])

Target text: enti sem ne yafet 
Input array shape: (34096,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train = train.map(prepare_dataset, remove_columns=train.column_names, batch_size=8, num_proc=1, batched=True)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
test = test.map(prepare_dataset, remove_columns=test.column_names, batch_size=8, num_proc=1, batched=True)

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
# Save the processed datasets
train.save_to_disk("/content/drive/MyDrive/Twi_ASR/1_Mose_PD/train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/885 [00:00<?, ? examples/s]

In [ ]:
test.save_to_disk("/content/drive/MyDrive/Twi_ASR/1_Mose_PD/test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

# Load the datasets back
train = load_from_disk("/content/drive/MyDrive/Twi_ASR/PD_21237:33102_2688:4033/train_dataset")
test = load_from_disk("/content/drive/MyDrive/Twi_ASR/PD_21237:33102_2688:4033/test_dataset")

**Setting-up Trainer**

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from evaluate import load

wer_metric = load("wer")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import numpy as np


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    #"facebook/wav2vec2-xls-r-300m",
    "/content/drive/MyDrive/Twi_ASR/wav2vec2-large-xlsr-twi_PD/checkpoint-21200",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2178: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/Twi_ASR/wav2vec2-large-xlsr-twi_PD",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)


<ipython-input-17-23c800d7e7f4>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Free GPU memory
torch.cuda.empty_cache()

# Check current memory usage
torch.cuda.memory_summary()

Training

In [ ]:
#trainer.train()
trainer.train(resume_from_checkpoint=True)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3083: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this

Step,Training Loss,Validation Loss,Wer
21600,0.026800,0.328441,0.378037
22000,0.083800,0.290398,0.382003


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in 

TrainOutput(global_step=22260, training_loss=0.0030326700167728777, metrics={'train_runtime': 1623.6139, 'train_samples_per_second': 219.233, 'train_steps_per_second': 13.71, 'total_flos': 3.766517570791279e+19, 'train_loss': 0.0030326700167728777, 'epoch': 30.0})